In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/caba_para_mapa.csv',error_bad_lines=False)

#tenia un nan nose porque
properati.dropna(inplace=True)

<IPython.core.display.Javascript object>

In [2]:
cant_buckets = 15

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

#tenia un nan nose porque
properati.dropna(inplace=True)

In [3]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

scaler = StandardScaler()
X = scaler.fit_transform(X,y)
#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

True
True


### Busco una aproximacion de hiper-parametros con random search

In [4]:
%%notify

#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

sgd = SGDClassifier(n_jobs=-1)


alpha=np.arange(0.000001,1,0.000001)
pen =['l2','l1','elasticnet']
loss = ["hinge", "log", "modified_huber", "squared_hinge", "perceptron", "squared_loss", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"]
param_grid = {"loss": loss, "penalty": pen, "alpha": alpha}
custom_cv = ShuffleSplit(n_splits=5, test_size=0.2)

# run randomized search
random_search = RandomizedSearchCV(sgd, param_distributions=param_grid,
n_iter=200,cv=custom_cv)
start = time()
random_search.fit(X, y)

print("RandomizedSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(random_search.cv_results_['params'])))
print("")
score.report_single(random_search.cv_results_)

/home/ale/.local/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


RandomizedSearchCV duro 255.65 segundos para 200 candidatos a hyper-parametros.

Puesto: 1
Promedio training score: 0.279 (std: 0.010)
Promedio validation score: 0.282 (std: 0.013)
Promedio score time: 0.006s
Hiper-parametros: {'penalty': 'elasticnet', 'loss': 'log', 'alpha': 0.61665400000000004}

Puesto: 2
Promedio training score: 0.274 (std: 0.003)
Promedio validation score: 0.280 (std: 0.013)
Promedio score time: 0.007s
Hiper-parametros: {'penalty': 'l1', 'loss': 'modified_huber', 'alpha': 0.82214100000000001}

Puesto: 2
Promedio training score: 0.274 (std: 0.003)
Promedio validation score: 0.280 (std: 0.013)
Promedio score time: 0.016s
Hiper-parametros: {'penalty': 'l1', 'loss': 'modified_huber', 'alpha': 0.83802699999999997}

Puesto: 2
Promedio training score: 0.274 (std: 0.003)
Promedio validation score: 0.280 (std: 0.013)
Promedio score time: 0.017s
Hiper-parametros: {'penalty': 'l1', 'loss': 'modified_huber', 'alpha': 0.97244900000000001}

Puesto: 2
Promedio training score: 0.2

/home/ale/.local/lib/python2.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ale/.local/lib/python2.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('std_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


<IPython.core.display.Javascript object>

In [20]:
%%notify

#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

sgd = SGDClassifier(n_jobs=-1)
        
alpha=np.arange(0.01,1,0.01)
pen =['l2','elasticnet',"l1"]
loss = ["log", "modified_huber","huber"]
param_grid = {"loss": loss, "penalty": pen, "alpha": alpha}

custom_cv = ShuffleSplit(n_splits=5, test_size=0.2)

grid_search = GridSearchCV(sgd,param_grid=param_grid,cv=custom_cv)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(grid_search.cv_results_['params'])))
score.report_single(grid_search.cv_results_)

GridSearchCV took 620.32 seconds for 891 candidate parameter settings.
Model with rank: 1
Mean training score: 0.282 (std: 0.008)
Mean validation score: 0.291 (std: 0.031)
Parameters: {'penalty': 'elasticnet', 'alpha': 0.65000000000000002, 'loss': 'log'}

Model with rank: 2
Mean training score: 0.282 (std: 0.011)
Mean validation score: 0.291 (std: 0.019)
Parameters: {'penalty': 'l2', 'alpha': 0.37, 'loss': 'log'}

Model with rank: 3
Mean training score: 0.282 (std: 0.009)
Mean validation score: 0.290 (std: 0.037)
Parameters: {'penalty': 'l2', 'alpha': 0.94000000000000006, 'loss': 'log'}

Model with rank: 4
Mean training score: 0.279 (std: 0.004)
Mean validation score: 0.290 (std: 0.030)
Parameters: {'penalty': 'elasticnet', 'alpha': 0.42999999999999999, 'loss': 'log'}

Model with rank: 5
Mean training score: 0.282 (std: 0.014)
Mean validation score: 0.289 (std: 0.019)
Parameters: {'penalty': 'elasticnet', 'alpha': 0.90000000000000002, 'loss': 'log'}

Model with rank: 6
Mean training sc

<IPython.core.display.Javascript object>